## 测试 mci_gru 框架

In [5]:
from main import *

In [ ]:
def make_dataset(F588080_temp, F159915_temp, seq_len, seq_gap, factor_list, label_name):
    data = []
    label = []
    for Fund_temp in [F588080_temp, F159915_temp]:
        Fund_temp = Fund_temp[Fund_temp.index.strftime('%H:%M')<='14:00']
        for i in tqdm(range(seq_len, len(Fund_temp), seq_gap)):
            temp_data = Fund_temp.iloc[i-seq_len:i][factor_list].values
            temp_label = Fund_temp.iloc[i-1][label_name]
            if np.isnan(temp_data).sum()+np.isnan(temp_label) == 0:
                data.append(temp_data)
                label.append(temp_label)
    return np.array(data), np.array(label)

In [9]:
F588080_Train

,windcode,open,high,low,close,volume,amount,change,pctchange,htc_ret
2022-02-14 09:30:00,588080.SH,1.200,1.200,1.190,1.190,4578800.0,5473856.0,-0.014,-0.011628,0.013434
2022-02-14 09:31:00,588080.SH,1.191,1.191,1.185,1.186,3978803.0,4719635.0,-0.004,-0.003361,0.017707
2022-02-14 09:32:00,588080.SH,1.186,1.194,1.185,1.193,2997000.0,3566074.0,0.007,0.005902,0.011735
2022-02-14 09:33:00,588080.SH,1.193,1.198,1.193,1.198,2019600.0,2412967.0,0.005,0.004191,0.006672
2022-02-14 09:34:00,588080.SH,1.199,1.202,1.198,1.202,3673700.0,4409735.0,0.004,0.003339,0.004160
...,...,...,...,...,...,...,...,...,...,...
2023-12-29 14:55:00,588080.SH,0.873,0.873,0.872,0.872,1099900.0,959909.0,0.000,0.000000,-0.001145
2023-12-29 14:56:00,588080.SH,0.873,0.873,0.872,0.872,1330800.0,1161037.0,0.000,0.000000,0.000000
2023-12-29 14:57:00,588080.SH,0.872,0.873,0.872,0.872,1802407.0,1572105.0,0.000,0.000000,-0.001145
2023-12-29 14:58:00,588080.SH,0.873,0.873,0.872,0.873,2035000.0,1776124.0,0.001,0.001147,0.000000


In [8]:
with open('test1.yaml', 'r') as file:
    args = DictToObj(yaml.safe_load(file))
ETF_minquotes = pd.read_parquet('ETF_minquotes.parquet')
F588080 = ETF_minquotes.loc[ETF_minquotes['windcode'] == '588080.SH']
F159915 = ETF_minquotes.loc[ETF_minquotes['windcode'] == '159915.SZ']
F588080.loc[:,'htc_ret'] = F588080.groupby(F588080.index.date).apply(lambda x:x['close'].iloc[-1]/x['open'].shift(-1)-1).values
F159915.loc[:,'htc_ret'] = F159915.groupby(F159915.index.date).apply(lambda x:x['close'].iloc[-1]/x['open'].shift(-1)-1).values

F588080_Train = F588080.loc[(F588080.index.date >= args.TRAIN.START_DATE) & (F588080.index.date <= args.TRAIN.END_DATE)]
F159915_Train = F159915.loc[(F159915.index.date >= args.TRAIN.START_DATE) & (F159915.index.date <= args.TRAIN.END_DATE)]
F588080_Valid = F588080.loc[(F588080.index.date >= args.VALID.START_DATE) & (F588080.index.date <= args.VALID.END_DATE)]
F159915_Valid = F159915.loc[(F159915.index.date >= args.VALID.START_DATE) & (F159915.index.date <= args.VALID.END_DATE)]
F588080_Test = F588080.loc[(F588080.index.date >= args.TEST.START_DATE) & (F588080.index.date <= args.TEST.END_DATE)]
F159915_Test = F159915.loc[(F159915.index.date >= args.TEST.START_DATE) & (F159915.index.date <= args.TEST.END_DATE)]

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_27052\2034361835.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  F588080.loc[:,'htc_ret'] = F588080.groupby(F588080.index.date).apply(lambda x:x['close'].iloc[-1]/x['open'].shift(-1)-1).values
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_27052\2034361835.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  F159915.loc[:,'htc_ret'] = F159915.groupby(F159915.index.date).apply(lambda x:x['close'].iloc[-1]/x['open'].shift(-1)-1).values


In [ ]:
class AGRU(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(AGRU, self).__init__()
        self.hidden_size = hidden_size

        # 输入到隐藏层的权重
        self.weight_ih = nn.Parameter(torch.randn(3 * hidden_size, input_size))
        # 隐藏层到隐藏层的权重
        self.weight_hh = nn.Parameter(torch.randn(3 * hidden_size, hidden_size))
        # 偏置项
        self.bias_ih = nn.Parameter(torch.randn(3 * hidden_size))
        self.bias_hh = nn.Parameter(torch.randn(3 * hidden_size))

    def forward(self, input):
        """
        input: (seq_len, batch_size, input_size)
        h_prev: (batch_size, hidden_size)
        """
        seq_len, batch_size, input_size = input.shape
        hidden_size = self.hidden_size
        h_prev = torch.zeros(batch_size, hidden_size)
        # 初始化输出和隐藏状态
        outputs = []
        h_next = h_prev

        for t in range(seq_len):
            x_t = input[t]  # (batch_size, input_size)

            # 计算输入和隐藏状态的线性变换
            gates_ih = torch.mm(x_t, self.weight_ih.t()) + self.bias_ih  # (batch_size, 3 * hidden_size)
            gates_hh = torch.mm(h_next, self.weight_hh.t()) + self.bias_hh  # (batch_size, 3 * hidden_size)

            # 拆分线性变换的结果
            r_gate, z_gate, n_gate = gates_ih.chunk(3, 1)  # 每部分形状: (batch_size, hidden_size)
            r_gate_h, z_gate_h, n_gate_h = gates_hh.chunk(3, 1)

            # 计算重置门、更新门和候选隐藏状态
            r_gate = torch.sigmoid(r_gate + r_gate_h)
            z_gate = torch.sigmoid(z_gate + z_gate_h)
            n_gate = torch.tanh(n_gate + r_gate * n_gate_h)

            # 更新隐藏状态
            h_next = (1 - z_gate) * n_gate + z_gate * h_next

            # 保存当前时间步的隐藏状态
            outputs.append(h_next.unsqueeze(0))  # (1, batch_size, hidden_size)

        # 将所有时间步的隐藏状态拼接起来
        outputs = torch.cat(outputs, dim=0)  # (seq_len, batch_size, hidden_size)

        # 返回所有时间步的输出和最后一步的隐藏状态
        return outputs, h_next